In [2]:
import ee
import geemap
import pandas as pd
import altair as alt
import numpy as np
import folium

In [3]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [4]:
# def functions
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function

In [5]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

In [6]:
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-38, 'years'), today)
ppt = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').filterDate(date_range).select('ppt')
aoi = ee.FeatureCollection('TIGER/2016/Counties').filter(
    ee.Filter.eq('NAME', 'Coconino')).geometry()

In [7]:
# reduce data
reduce_ppt = create_reduce_region_function(
    geometry=aoi, reducer=ee.Reducer.mean(), scale=5000, crs='EPSG:4326')

ppt_stat_fc = ee.FeatureCollection(ppt.map(reduce_ppt)).filter(
    ee.Filter.notNull(ppt.first().bandNames()))

In [8]:
ppt_dict = fc_to_dict(ppt_stat_fc).getInfo()

In [9]:
print(type(ppt_dict), '\n')
for prop in ppt_dict.keys():
    print(prop + ':', ppt_dict[prop][0:3] + ['...'])

<class 'dict'> 

millis: [449625600000, 452217600000, 454896000000, '...']
ppt: [11.534907018713827, 3.2184749650250235, 8.493474191996972, '...']
system:index: ['198404', '198405', '198406', '...']


In [10]:
ppt_df = pd.DataFrame(ppt_dict)

In [11]:
display(ppt_df)
print(ppt_df.dtypes)

,millis,ppt,system:index
0,449625600000,11.534907,198404
1,452217600000,3.218475,198405
2,454896000000,8.493474,198406
3,457488000000,79.038749,198407
4,460166400000,65.461223,198408
...,...,...,...
450,1633046400000,33.219086,202110
451,1635724800000,1.284263,202111
452,1638316800000,50.511307,202112
453,1640995200000,8.675930,202201


millis            int64
ppt             float64
system:index     object
dtype: object


In [12]:
# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

In [13]:
ppt_df = add_date_info(ppt_df)
ppt_df.head(5)

,millis,ppt,system:index,Timestamp,Year,Month,Day,DOY
0,449625600000,11.534907,198404,1984-04-01,1984,4,1,92
1,452217600000,3.218475,198405,1984-05-01,1984,5,1,122
2,454896000000,8.493474,198406,1984-06-01,1984,6,1,153
3,457488000000,79.038749,198407,1984-07-01,1984,7,1,183
4,460166400000,65.461223,198408,1984-08-01,1984,8,1,214


In [14]:
ppt_df.dtypes

millis                   int64
ppt                    float64
system:index            object
Timestamp       datetime64[ns]
Year                     int64
Month                    int64
Day                      int64
DOY                      int64
dtype: object

In [ ]:
# ppt_df.to_csv('~/Downloads/XXXXXX.csv')

In [18]:
alt.Chart(ppt_df).mark_bar(size=1).encode(
    x='Timestamp:T',
    y='ppt:Q',
    color=alt.Color(
        'ppt:Q', scale=alt.Scale(scheme='redblue', domain=(-5, 5))),
    tooltip=[
        alt.Tooltip('Timestamp:T', title='Date'),
        alt.Tooltip('ppt:Q', title='ppt')
    ]).properties(width=600, height=300)

alt.Chart(...)

In [19]:
alt.Chart(ppt_df).mark_rect().encode(
    x='Year:O',
    y='Month:O',
    color=alt.Color(
        'mean(ppt):Q', scale=alt.Scale(scheme='redblue', domain=(-5, 5))),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Month:O', title='Month'),
        alt.Tooltip('mean(ppt):Q', title='ppt')
    ]).properties(width=600, height=300)

alt.Chart(...)